---
title: "Data"
author: "[Jason Foster](mailto:jason.j.foster@gmail.com)"
date: last-modified
categories:
  - analysis
  - finance
  - python
draft: true
execute:
  eval: false
  freeze: false
editor: 
  mode: source
---

In [ ]:
# !pip install --index-url=https://blpapi.bloomberg.com/repository/releases/python/simple/ blpapi
import blpapi
import pandas as pd

# Static query

-   <https://www.bloomberg.com/professional/support/api-library/>
-   Page 80: <https://bloomberg.github.io/blpapi-docs/>

In [ ]:
def create_session(host = "localhost", port = 8194):
  
  options = blpapi.SessionOptions()
  options.setServerHost(host)
  options.setServerPort(port)
  
  session = blpapi.Session(options)
  session.start()
  session.openService("//blp/refdata")
  
  return session

In [ ]:
def process_response(event, fields, start_date = None, end_date = None):
  
  data_ls = []
  
  for msg in event:
    
    security_data = msg.getElement("securityData")
    
    if (start_date):
      
      security = security_data.getElementAsString("security")
      field_data = security_data.getElement("fieldData")
      
      for x in field_data:
        
        record = {"security": security, "date": x.getElementAsDatetime("date")}
        
        for field in fields:
          try:
            record[field] = x.getElement(field).getValue()
          except:
            pass
        
        data_ls.append(record)
        
    else:
      
      for x in security_data.values():
        
        security = x.getElementAsString("security")
        field_data = x.getElement("fieldData")
        record = {"security": security}
        
        for field in fields:
          try:
            record[field] = field_data.getElement(field).getValue()
          except:
            pass
        
        data_ls.append(record)
    
  return pd.DataFrame(data_ls)

# Data point

In [ ]:
def bdp(securities, fields):
  
  if isinstance(securities, str):
    securities = [securities]
  
  if isinstance(fields, str):
    fields = [fields]
  
  session = create_session()
  service = session.getService("//blp/refdata")
  request = service.createRequest("ReferenceDataRequest")
  
  for security in securities:
    request.append("securities", security)
  
  for field in fields:
    request.append("fields", field)
    
  session.sendRequest(request)
  
  status = True
  result = pd.DataFrame()
  
  while status:
    
    event = session.nextEvent()
    
    if (event.eventType() in [blpapi.Event.RESPONSE, blpapi.Event.PARTIAL_RESPONSE]):
      result = pd.concat([result, process_response(event, fields)], ignore_index = True)
      
    if (event.eventType() == blpapi.Event.RESPONSE):
      status = False
  
  session.stop()
  
  return result

In [ ]:
securities = ["IBM US Equity", "GOOG US Equity", "MSFT US Equity", "BA US Equity"]
fields = ["MARKET_SECTOR_DES", "GICS_SECTOR_NAME", "ID_CUSIP", "PX_LAST"]

In [ ]:
#| eval: false
bdp_df = bdp(securities, fields)
print(bdp_df)

# Data history

In [ ]:
def bdh(securities, fields, start_date, end_date = None):
  
  if isinstance(securities, str):
    securities = [securities]
  
  if isinstance(fields, str):
    fields = [fields]
  
  session = create_session()
  service = session.getService("//blp/refdata")
  request = service.createRequest("HistoricalDataRequest")
  
  for security in securities:
    request.append("securities", security)
    
  for field in fields:
    request.append("fields", field)

  request.set("startDate", start_date)
  
  if (end_date):
    request.set("endDate", end_date)
    
  request.set("nonTradingDayFillOption", "ALL_CALENDAR_DAYS") # "NON_TRADING_WEEKDAYS", "ALL_CALENDAR_DAYS", "ACTIVE_DAYS_ONLY"
  request.set("nonTradingDayFillMethod", "NIL_VALUE") # "PREVIOUS_VALUE", "NIL_VALUE" 
  
  session.sendRequest(request)
  
  status = True
  result = pd.DataFrame()
  
  while status:
    
    event = session.nextEvent()
    
    if (event.eventType() in [blpapi.Event.RESPONSE, blpapi.Event.PARTIAL_RESPONSE]):
      result = pd.concat([result, process_response(event, fields, start_date, end_date)], ignore_index = True)
    
    if (event.eventType() == blpapi.Event.RESPONSE):
      status = False
      
  session.stop()
  
  return result

In [ ]:
fields = ["PX_LAST", "PX_BID", "PX_ASK"]
start_date = "20231201"
end_date = "20231205"

In [ ]:
#| eval: false
bdh_df = bdh(securities, fields, start_date, end_date)
print(bdh_df)